# Subgroups and Lagrange's Theorem

**Module 01e** | Modular Arithmetic and Groups

*Some elements can't reach the whole group, but the little group they DO reach is surprisingly constrained.*

> **Where we left off.** In [01d](01d-cyclic-groups-generators.ipynb) we discovered that $g = 2$ in $(\mathbb{Z}/7\mathbb{Z}^*, \times)$ only generates $\{1, 2, 4\}$, three elements out of six. Meanwhile $g = 3$ reaches all six. But here's a question we didn't answer: **is $\{1, 2, 4\}$ itself a group?** And could it have been size 5 instead of size 3? What sizes are even *possible*?

## Objectives

By the end of this notebook you will be able to:

1. Recognize a **subgroup** by checking the group axioms on a subset.
2. State **Lagrange's theorem**: a subgroup's size must divide the group's size.
3. Explain **why** it works using cosets (shifted copies of the subgroup).
4. Apply Lagrange's theorem to predict which element orders are possible.

## A Group Inside a Group

Let's pick up right where [01d](01d-cyclic-groups-generators.ipynb) left off. In $(\mathbb{Z}/7\mathbb{Z}^*, \times)$, element 2 generates the set $\{1, 2, 4\}$. Is this a genuine group under multiplication mod 7?

We need the four axioms from [01c](01c-groups-first-look.ipynb): closure, associativity, identity, inverses.

In [ ]:
# Is {1, 2, 4} a group under multiplication mod 7?
p = 7
S = [1, 2, 4]

# Closure: multiply every pair, check the result stays in S
print('Closure check (multiplication table):')
all_closed = True
for a in S:
    row = []
    for b in S:
        product = (a * b) % p
        row.append(product)
        if product not in S:
            all_closed = False
    print(f'  {a} * {S} = {row}')
print(f'  All products land in {{1, 2, 4}}? {all_closed}')

# Identity: 1 is in S
print(f'\nIdentity: 1 is in S? {1 in S}')

# Inverses: for each element, find its inverse
print(f'\nInverses:')
for a in S:
    inv = Mod(a, p)^(-1)
    print(f'  {a}^(-1) = {inv}   in S? {int(inv) in S}')

# Associativity: inherited from Z/7Z* (always holds for modular multiplication)
print(f'\nAssociativity: inherited from the bigger group (always holds).')
print(f'\nVerdict: {{1, 2, 4}} IS a group under multiplication mod 7!')

This smaller group living inside the bigger one has a name: it's a **subgroup**.

> **Definition.** A subset $H \subseteq G$ is a **subgroup** of $G$ (written $H \leq G$) if $H$ is itself a group under the same operation.

Two subgroups always exist for free:
- The **trivial subgroup** $\{e\}$ (just the identity), here $\{1\}$.
- The **whole group** $G$ (every group is a subgroup of itself).

The interesting subgroups are the ones in between.

## What Sizes Are Possible?

$(\mathbb{Z}/7\mathbb{Z}^*, \times)$ has 6 elements. The subgroup $\{1, 2, 4\}$ has 3 elements. Could we find a subgroup of size 4? Or size 5?

Let's generate the subgroup $\langle g \rangle$ for every element $g$ and see what sizes appear.

In [ ]:
# Generate <g> for every g in (Z/7Z*, ×)
p = 7
print(f'Subgroups of (Z/{p}Z*, ×):     [group size = {p-1}]\n')

seen_sizes = set()
for g in range(1, p):
    # Compute g, g^2, g^3, ... until we hit 1
    subgroup = []
    val = 1
    for _ in range(p):
        val = (val * g) % p
        subgroup.append(val)
        if val == 1:
            break
    seen_sizes.add(len(subgroup))
    print(f'  <{g}> = {subgroup}   (size {len(subgroup)})')

print(f'\nSubgroup sizes that appear: {sorted(seen_sizes)}')
print(f'Divisors of 6: {divisors(6)}')
print(f'\nNotice: every subgroup size divides the group size 6.')
print(f'Sizes 4 and 5 are NOT divisors of 6, so they CANNOT appear.')

Subgroup sizes 1, 2, 3, 6. Group size 6. Every subgroup size divides 6.

Is this a coincidence? Let's try a bigger group to find out.

## More Evidence: Additive Groups

$(\mathbb{Z}/12\mathbb{Z}, +)$ has 12 elements, so 12 has many divisors: 1, 2, 3, 4, 6, 12. This gives us more room to spot the pattern.

In an additive group, the subgroup generated by $a$ is $\langle a \rangle = \{a, 2a, 3a, \ldots\}$, keep adding $a$ until you cycle back to 0.

In [ ]:
# Generate <a> for every a in (Z/12Z, +)
n = 12

def generate_subgroup_additive(a, n):
    """Compute the subgroup <a> in (Z/nZ, +)."""
    subgroup = set()
    x = 0
    while True:
        x = (x + a) % n
        subgroup.add(x)
        if x == 0:
            break
    return sorted(subgroup)

print(f'Subgroups of (Z/{n}Z, +):     [group size = {n}]\n')

# Collect distinct subgroups
seen = {}  # frozenset -> (generator, subgroup)
for a in range(n):
    sg = generate_subgroup_additive(a, n)
    key = frozenset(sg)
    if key not in seen:
        seen[key] = (a, sg)

for key in sorted(seen.keys(), key=len):
    gen, sg = seen[key]
    print(f'  <{gen}> = {sg}   (size {len(sg)})')

sizes = sorted(set(len(sg) for _, sg in seen.values()))
print(f'\nSubgroup sizes: {sizes}')
print(f'Divisors of {n}: {divisors(n)}')
print(f'\nAgain: every subgroup size divides the group size. Not a coincidence.')

> **Checkpoint.** Before reading on, try to predict: in $(\mathbb{Z}/15\mathbb{Z}, +)$, what subgroup sizes are possible? (Hint: divisors of 15 are 1, 3, 5, 15.)

## Lagrange's Theorem

This pattern has been proven true for ALL finite groups, not just our examples.

> **Lagrange's Theorem.** If $H$ is a subgroup of a finite group $G$, then $|H|$ divides $|G|$.

That's it. Subgroup sizes must divide the group size. No exceptions, ever.

This is arguably the most important theorem in elementary group theory. But *why* is it true? The answer involves a beautiful idea: **cosets**.

## Why It Works: Cosets

Here's the key idea. Take a subgroup $H$ and "slide" it around the group. Each slide produces a **coset**.

Concretely, in $(\mathbb{Z}/12\mathbb{Z}, +)$ with $H = \langle 4 \rangle = \{0, 4, 8\}$:

- Slide by 0: $0 + H = \{0, 4, 8\}$
- Slide by 1: $1 + H = \{1, 5, 9\}$
- Slide by 2: $2 + H = \{2, 6, 10\}$
- Slide by 3: $3 + H = \{3, 7, 11\}$

Slide by 4? That gives $\{4, 8, 0\}$, the same as the first coset. We've already covered everything.

Think of it like **dealing cards into piles**. Each pile (coset) has the same number of cards as $H$. The piles never overlap. Together they use up every card in the deck.

In [ ]:
# Compute cosets of H = <4> = {0, 4, 8} in Z/12Z
n = 12
H = generate_subgroup_additive(4, n)
print(f'Subgroup H = <4> = {H}   (size {len(H)})\n')

# Build distinct cosets
covered = set()
cosets = []
for a in range(n):
    if a in covered:
        continue
    coset = sorted([(a + h) % n for h in H])
    cosets.append((a, coset))
    covered.update(coset)

for rep, coset in cosets:
    print(f'  {rep} + H = {coset}')

print(f'\n4 cosets, each of size 3, covering all 12 elements.')
print(f'12 = 3 x 4   (group size = subgroup size x number of cosets)')
print(f'\nSince 12 = |H| x (number of cosets), |H| must divide 12. That\'s Lagrange!')

The argument works for ANY subgroup of ANY finite group:

1. Every coset has the **same size** as $H$ (sliding doesn't change the count).
2. Cosets **never overlap** (if two cosets share an element, they're identical).
3. Cosets **cover everything** (every element $a$ lives in the coset $a + H$).

So the group is partitioned into equal-sized pieces: $|G| = |H| \times (\text{number of cosets})$. Since both sides are integers, $|H|$ must divide $|G|$. Done.

The number of cosets is called the **index** of $H$ in $G$, written $[G:H]$.

## Seeing Cosets

Let's color the elements of $\mathbb{Z}/12\mathbb{Z}$ by which coset they belong to. This makes the partition visible.

In [ ]:
n = 12
H = generate_subgroup_additive(4, n)

# Assign each element to its coset
coset_colors = ['royalblue', 'orangered', 'forestgreen', 'mediumorchid']
element_colors = {}
coset_list = []
covered = set()
coset_idx = 0
for a in range(n):
    if a in covered:
        continue
    coset = [(a + h) % n for h in H]
    for elem in coset:
        element_colors[elem] = coset_colors[coset_idx]
    coset_list.append((a, coset))
    covered.update(coset)
    coset_idx += 1

# Draw elements on a circle, colored by coset
G = Graphics()
angles = [2 * pi * i / n - pi/2 for i in range(n)]
r = 1.0

for i in range(n):
    x = r * cos(angles[i])
    y = r * sin(angles[i])
    G += circle((x, y), 0.12, fill=True, facecolor=element_colors[i],
                edgecolor='white', linewidth=2, zorder=3)
    G += text(str(i), (x, y), fontsize=11, fontweight='bold',
              color='white', zorder=4)

# Faint outline circle
G += parametric_plot((r * cos(t), r * sin(t)), (t, 0, 2*pi),
                      color='lightgray', thickness=0.5, zorder=1)

G.axes(False)
G.set_aspect_ratio(1)
G.show(figsize=5, title='Z/12Z colored by cosets of H = {0, 4, 8}')

# Print the legend since SageMath plot titles don't support rich legends easily
for idx, (rep, coset) in enumerate(coset_list):
    print(f'  {rep} + H = {coset}')
print('\nSame color = same coset. Four colors, three elements each, no overlap.')

The pattern is striking: the cosets are evenly spaced around the circle. Each coset is a "rotated copy" of $H$.

> **Checkpoint.** What happens if we use $H = \langle 3 \rangle = \{0, 3, 6, 9\}$ instead? How many cosets would there be, and what size? (Answer: $12/4 = 3$ cosets of size 4.)

## Consequences for Element Orders

Since $\langle a \rangle$ is always a subgroup, and $\text{ord}(a) = |\langle a \rangle|$, Lagrange immediately tells us:

> **Corollary.** The order of every element must divide the size of the group.

In a group of size 12, the only possible element orders are 1, 2, 3, 4, 6, 12. An element of order 5 or 7 **cannot exist**.

In [ ]:
# Element orders in (Z/12Z, +) must divide 12
n = 12
print(f'Group: (Z/{n}Z, +),  |G| = {n}')
print(f'Divisors of {n}: {divisors(n)}')
print(f'So element orders can ONLY be: {divisors(n)}\n')

for a in range(n):
    if a == 0:
        order = 1
    else:
        order = n // gcd(a, n)
    print(f'  ord({a}) = {order}')

actual_orders = sorted(set(
    1 if a == 0 else n // gcd(a, n) for a in range(n)
))
print(f'\nOrders that actually appear: {actual_orders}')
print(f'Every single divisor of 12 shows up. (This always happens in cyclic groups.)')

## Back to Multiplication: A Surprise

Let's apply Lagrange to $(\mathbb{Z}/15\mathbb{Z}^*, \times)$. This group has $\varphi(15) = 8$ elements, so Lagrange says element orders can only be divisors of 8: that's 1, 2, 4, or 8.

Let's check.

In [ ]:
# Element orders in (Z/15Z*, ×)
n = 15
phi_n = euler_phi(n)
units = [a for a in range(1, n) if gcd(a, n) == 1]

print(f'Group: (Z/{n}Z*, ×),  |G| = phi({n}) = {phi_n}')
print(f'Divisors of {phi_n}: {divisors(phi_n)}')
print(f'So possible element orders: {divisors(phi_n)}\n')

actual_orders = set()
for a in units:
    order = Mod(a, n).multiplicative_order()
    actual_orders.add(order)
    print(f'  ord({a}) = {order}')

print(f'\nOrders that actually appear: {sorted(actual_orders)}')
missing = set(divisors(phi_n)) - actual_orders
if missing:
    print(f'Missing: {sorted(missing)}')
    print(f'\nNo element has order 8! That means no single element generates the')
    print(f'whole group. (Z/{n}Z*, ×) is NOT cyclic.')

This is an important subtlety:

> **Common mistake.** "Lagrange says every divisor of $|G|$ appears as a subgroup size." **No!** Lagrange only goes one direction: subgroup sizes *must be* divisors. It does NOT promise that every divisor appears. For cyclic groups (like $\mathbb{Z}/n\mathbb{Z}$) every divisor does give a subgroup, but for non-cyclic groups some divisors can be missing.

## Why This Matters for Cryptography

Lagrange's theorem explains why cryptographers care about **safe primes**.

If $p$ is prime, $(\mathbb{Z}/p\mathbb{Z}^*)$ has order $p - 1$. The subgroup sizes are the divisors of $p - 1$. If $p - 1$ has lots of small factors, there are lots of small subgroups, and an attacker can exploit them (you'll see this in the [smooth-order attack](../break/smooth-order-attack.ipynb) notebook).

A **safe prime** is $p = 2q + 1$ where $q$ is also prime. Then $p - 1 = 2q$, and its only divisors are $1, 2, q, 2q$. The only subgroups have size 1, 2, $q$, or $2q$. No small subgroups to exploit.

In [ ]:
# Compare: regular prime vs safe prime
print('Regular prime p = 41:  p-1 = 40')
print(f'  Divisors of 40: {divisors(40)}')
print(f'  That\'s {len(divisors(40))} possible subgroup sizes. Lots of small ones!\n')

print('Safe prime p = 47:  p-1 = 46 = 2 × 23')
print(f'  Divisors of 46: {divisors(46)}')
print(f'  Only {len(divisors(46))} possible subgroup sizes. Much harder to attack.')

## Exercises

### Exercise 1 (Worked)

Find all subgroups of $(\mathbb{Z}/7\mathbb{Z}^*, \times)$. For each, list its elements and verify its size divides 6.

In [ ]:
# Exercise 1 (Worked): All subgroups of (Z/7Z*, ×)
p = 7
print(f'Group: (Z/{p}Z*, ×),  |G| = {p-1}')
print(f'Divisors of {p-1}: {divisors(p-1)}\n')

# Generate <g> for each g, collect distinct subgroups
seen = {}
for g in range(1, p):
    subgroup = []
    val = 1
    for _ in range(p):
        val = (val * g) % p
        subgroup.append(val)
        if val == 1:
            break
    key = frozenset(subgroup)
    if key not in seen:
        seen[key] = (g, sorted(subgroup))

for key in sorted(seen.keys(), key=len):
    gen, sg = seen[key]
    divides = (p - 1) % len(sg) == 0
    print(f'  <{gen}> = {sg}   size {len(sg)}   divides 6? {"yes" if divides else "NO"}')

print(f'\n{len(seen)} subgroups total. Sizes 1, 2, 3, 6 = all divisors of 6.')
print(f'(Z/7Z* is cyclic, so every divisor appears.)')

### Exercise 2 (Guided)

Compute the cosets of $H = \langle 5 \rangle$ in $(\mathbb{Z}/15\mathbb{Z}, +)$. How many cosets are there? Verify they partition $\{0, 1, \ldots, 14\}$.

In [ ]:
# Exercise 2: Cosets of <5> in (Z/15Z, +)
n = 15

# Step 1: compute H = <5>
H = generate_subgroup_additive(5, n)
print(f'H = <5> = {H},  |H| = {len(H)}')
print(f'Predicted number of cosets: {n} / {len(H)} = {n // len(H)}\n')

# Step 2: TODO, compute cosets a + H for a = 0, 1, 2, ...
# until all 15 elements are covered
# Hint: for each a not yet covered, compute {(a + h) % n for h in H}

# Step 3: TODO, verify the cosets partition {0, 1, ..., 14}
# Check: no overlaps, and union = {0, ..., 14}

### Exercise 3 (Independent)

In $(\mathbb{Z}/31\mathbb{Z}^*, \times)$, the group has order 30.

1. List all possible subgroup sizes (= divisors of 30).
2. For each divisor $d$, find an element whose order is $d$.
3. Is there a divisor with no corresponding element? Is $(\mathbb{Z}/31\mathbb{Z}^*)$ cyclic?

In [ ]:
# Exercise 3: Your code here


## Summary

| Concept | Key idea |
|---------|----------|
| **Subgroup** | A subset that is itself a group under the same operation |
| **Lagrange's theorem** | $|H|$ divides $|G|$, always, no exceptions |
| **Cosets** | Shifted copies of $H$ that partition $G$ into equal-sized pieces |
| **Index** | $[G:H] = |G|/|H|$ = number of cosets |
| **Element orders** | $\text{ord}(a)$ must divide $|G|$ (because $\langle a \rangle$ is a subgroup) |

**The punchline:** group structure is not free-form. Subgroup sizes, element orders, and coset counts are all locked together by divisibility. This rigidity is exactly what makes groups useful for cryptography.

**Next:** [Visualizing Group Structure](01f-group-visualization.ipynb), where we'll draw Cayley graphs, subgroup lattices, and multiplication heatmaps to *see* everything we've computed.